POWERMAX_Tonka
Notebook created 03/30/23 from OptionGun03-Tonka-Prdtn
Last rev 08/04/23 - Replaced OptionGunLib01 with version from RB local drive. 
Uses full OptionGunLib01.py and full PowerMax01 Main

User Notes
1. This program needs a user key from Darqube.com, which it uses to get real-time stock prices.
   Go to the Darqube website get a user account and choose the free service. Then get your key.
   Find the Dar_key line just below the "User Parameters" block in the cell below.
   Enter it between the quotes on the line:  DAR_key = ''
   
   To test the program, run a short list of tickers. At the bottom of the next cell (cell #1), find this line: 
   #TICKERS = ('QCOM', 'GOOGL', 'META', 'NVDA')
   Delete the # at the start of the line. Now it will run with only these four tickers.
  
   When you want to run all tickers, type the # back at the start of the line.
   
   Occasionally one ticker hangs up, either in getting the price or getting the option string. When the program crashes, 
   read the last line before the crash. Getting price (or data) for XXXX." Simply delete the ticker XXXX from the ticker list. 
   
2. OptionGun produces these csv files:
     all_options_suffix.csv. This file contains every option available for every ticker in the list. 
     Put_STO_short_Tonka.csv. Best STO options that meet criteria and expiry < 50 days out.
     Put_STO_mid_Tonka.csv.   Best STO options that meet criteria and 50 days <= expiry < 150 days out.
     Put_STO_long_Tonka.csv   Best STO options that meet criteria and 150 days out <= expiry.
     
     Call_STO_short_Tonka.csv. Best STO options that meet criteria and expiry < 50 days out.
     Call_STO_mid_Tonka.csv.   Best STO options that meet criteria and 50 days <= expiry < 150 days out.
     Call_STO_long_Tonka.csv   Best STO options that meet criteria and 150 days out <= expiry.
     
     root_data.csv. This contains a list of all tickers, with real-time stock price at the time of execution.

3. The parameters just under the "User Parameters" header block can be changed.
    Add and remove ticker symbols from the TICKERS = list. Enclose each ticker Follow the format shown.
    HIDE_TICKERS. You may want to collect option info for a ticker, but not see that ticker in best_file.csv.
    Just put the ticker to be hidden in this list.
    
4. PowerMax needs about 2-8 seconds to get the option string for each ticker. For 220 tickers, it will take about 15-20 minutes.



In [1]:
# ****************
# USER ADJUSTABLE PARAMETERS
# *****************

DAR_key = ''

TICKERS = ('AAL', 'AAPL', 'ABBV', 'ABT', 'ACLS', 'ACRE', 'AFL', 'ADBE', 'AFYA', 'AIG', 'AMD', 'ALK', 'AMGN', 'AMP', 'AMR', 'AMT', \
           'BNTX', 'BRSP', 'BSM', 'CAG', 'CAT', 'CBRE', 'CCI', 'CEG', 'CF', 'CFR', 'CHPT', 'CL', 'CLX', 'CMA', 'CMCSA', 'CMG', 'CPA', \
           'AMZN', 'AOS', 'ARCC', 'ARES', 'ASC', 'ASRT', 'ASO', 'AVB', 'AVGO', 'AX', 'AZO', 'BA', 'BAC', 'BGSF', 'BIIB', \
           'CPT', 'CRM', 'CRWD', 'CSCO', 'CSX', 'CUBE', 'CVCO', 'CVS', 'CVX', 'DAC', 'DAL', 'DDOG', 'DHI', 'DLR', 'DOCU', 'DUK', \
           'DVN', 'EL', 'ENB', 'EPAM', 'EPD', 'EPR', 'ET', 'EXC', 'EXP', 'EXPE', 'EXR', 'FAF', 'FANG', 'FCNCA', 'FDX', 'FITB', 'FLNG', \
           'FMS', 'FNF', 'FSLY', 'FSR', 'FYBR', 'GBX', 'GD', 'GIS', 'GLNG', 'GOL', 'JBLU', 'GOOG', 'GOOGL', 'GS', 'GSL', 'HA', 'HD', \
           'HDSN', 'HIW', 'HON', 'HR', 'HRB', 'HUBG', 'IBM', 'INTC', 'IPI', 'IRM', 'JEF', 'JNJ', 'JPM', 'JXN', 'K', 'KBH', \
           'KEY', 'KO', 'LADR', 'LLY', 'LMT', 'LOW', 'LRCX', 'LUV', 'LYB', 'MA', 'MAN', 'MCD', 'MCK', 'MDB', 'META', 'MGA', 'MLM', \
           'MMM', 'MNSO', 'MOD', 'MOS', 'MPW', 'MRK', 'MRNA', 'MRVL', 'MS', 'MSFT', 'MTB', 'MTCH', 'MU', 'NFLX', 'NKE', \
           'NKTR', 'NLY', 'NMM', 'NOW', 'NRG', 'NUE', 'NVDA', 'NWG', 'NYCB', 'O', 'OKTA', 'ON', 'PAA', 'PANW', \
           'PB', 'PD', 'PEBO', 'PERI', 'PFE', 'PFG', 'PGR', 'PHM', 'PII', 'PINE', 'PLTR', 'PNC', 'PRU', 'PSA', 'PYPL', \
           'QCOM', 'RC', 'RILY', 'RITM', 'RIVN', 'ROST', 'T', 'RTX', 'SB', 'SBLK', 'SBUX', 'SCHW', 'SEDG', 'SKT', 'SMCI', \
           'SNOW', 'SO', 'SOFI', 'SPG', 'SPY', 'STAG', 'STLD', 'STRL', 'STWD', 'TBBK', 'TMUS', 'TSLA', 'TRV', 'TXN', 'TSCO', \
           'TSN', 'UAL', 'UBS', 'UHAL', 'UNP', 'UPS', 'URI', 'USAC', 'USB', 'V', 'VICI', 'VIR', 'VGT', 'VIST', \
           'VLO', 'VNOM', 'VOO', 'VPG', 'VRNA', 'WAL', 'WDC', 'WEC', 'WMT', 'WPC', 'WSM', 'WFC', 'XPEL', \
           'XOM', 'ZIM', 'ZION', 'ZS')



HIDE_TICKERS = ('NOW', 'PLTR', 'VNOM', 'RIVN', 'NKTR') # Keep these tickers for database, but do not show them in best_options, because you do not want to own them.  

#Short list for testing
#TICKERS = ('ET', 'EPD', 'MU', 'DVN')


In [2]:
# ****************
# PowerMax01 - 04/02/23.
# *****************
# Goal - PowerMax runs all four main functional blocks: TradeManager, ScoreKeeper, CrystalBall, and OptionGun. 
#
# 
#   ToDo - Move GetERDates to CrystalBall. Not part of OptionGun.
#          Delete extra columns from saved STO profiles. What went wrong with existing delete? Reconcile puts_all with Putsssa
#
#   04/04/23 - Runs, with warnings in Bullets_PSTO. Fix warnings.
#   04/02/23 - Created from OptionGun03-Bullets.
#
# *****************
#
# Import Libraries
#
# *****************

import csv
import copy
import json
import numpy as np
import os
import pandas as pd
import requests   # for http requests
import scipy
from   scipy import stats
from   scipy.stats import norm
import time
import yfinance as yf

import datetime
from   datetime import datetime
from   datetime import date
from   datetime import timedelta
#from   datetime import fromtimestamp

from   dateutil.relativedelta  import relativedelta
from   yahoo_earnings_calendar import YahooEarningsCalendar

import OptionGunLib01

#from OptionGunLib01 import GetERDates
from OptionGunLib01 import GetPrices
from OptionGunLib01 import GetOptions
from OptionGunLib01 import BuildOptionMetrics
from OptionGunLib01 import Bullets_STO

# *****************
#
# Flow Control
#
# CHECK 3 PARAMS:  TICKERS, FILE_ID_TAG, profile
#
# *****************

update_root_data       = True  #skip Darqube call for tests.
update_root_data_ER    = False  #skip call to yahoo calendar for tests. 
update_option_data     = True  #skip call to yfinance if testing code.
run_BuildOptionMetrics = update_option_data  #skip call to BuildOptionMetrics if no update of option data.


# *****************
#
# CONSTANTS
#
# *****************

print(len(TICKERS), ' tickers to analyze.')
FILE_ID_TAG       = 'Tonka'
DATE_FORMAT = '%Y-%m-%d %H:%M:%S'
DATE_TIME_FORMAT  = '%Y-%m-%d %H:%M:%S'
DATE_ONLY_FORMAT  = '%Y-%m-%d'

TICKER_DELAY  = 0.015  # seconds between tickers for data requests.
EXPIRY_DELAY  = 0.025
YEC_DELAY     = 0.75

MAX_DAYSOUT   = 500.0  # This constant controls how many expiry strings are downloaded. Use low # to exclude BTOS and reduce ext. calls. Hi # includes BTOS.
MIN_DAYSOUT   = -1.0   

FEE_SPREAD    = 0.75     # Fee in the bid/ask range.
DAYS_TO_CHECK_ER = (1,4) #Update ER dates from yahoo on these days of the month.

# *****************
#
# MAIN PROGAM STARTS HERE (next 2 cells).
# 1. If stock market closed and broker_report file in folder
#       run TradeLogManager code block in Main program.'
#       run ScoreKeeper code block in Main program.
# 2. If stock market closed run CrystalBall code block in Main program.
# 3, If stock market open run OptionGun code block in Main program.
#
# *****************


OptionGunLib01 runs like a top.
240  tickers to analyze.


In [3]:
# *****************
#
# TradeManager
#
# 1. If stock market closed and broker_report file in folder
#       run TradeLogManager code block in Main program.'
# 2. Run BrokerReporter if new broker_report file is in folder.
# 3. Run TradeLogger, which updates trades_open and trades_closed
# 4. Update holdings, cost basis, # contracts committed, # contracts available. All these data columns are in root_data.

# *****************
#
# ScoreKeeper
#
# 1. Run sll ScoreKeeper functions.
#
# *****************

# *****************
#
# CrystalBall
#
# 1. Update all ML features for all option profiles.
# 2. Run ScoreKeeper code block in Main program.
# 2. If stock market closed run CrystalBall code block in Main program.
# 3, If stock market open run OptionGun code block in Main program.
#
# *****************


In [ ]:
# *****************
#
#OptionGun
#
# 1. Call GetPrices to get real-time price data from Darqube write to root_data. 
# 2. Call GetERDates to get dates from Yahoo Earnings Calendar and write to root_data.
# 3. Call GetOptions to get options data from yfinance. Use "try" statements to skip over exp dates where call to yfinance failed.
# 4. Call BuildOptionMetrics to build additional metrics fields in each option record.
# 5. Call Bullets_STO to screen all_options for each STO profile.
# FUTURE TODO
# 6. Call Bullets_BTC to build BTC trades for each open STO trade.
# 7. Call BuyWrite to screen for BuyWrite trades.
# 
# *****************

root_data = pd.read_csv('root_data.csv')   #Get root_data price and ER info.
root_data.set_index('Ticker', inplace = True, drop = True)
print('just read root_data')
#Use Darqube for stock price data - both market and historical.

print('start Darqube access at:  ', datetime.now(), '  Done in ',len(TICKERS) * 1.0 / 200.0, ' minutes.')

if update_root_data:
    root_data = GetPrices(TICKERS, root_data, DAR_key)

print(datetime.now())

if update_root_data_ER:                # skip call to YEC for ER update
    if date.today().day in DAYS_TO_CHECK_ER:
        print('Updating ER dates.')
        root_data = GetERDates(root_data, TICKERS, YEC_DELAY)

root_data.to_csv('root_data.csv')

# Get option chains
if update_option_data:
    all_options = GetOptions(root_data, TICKERS, EXPIRY_DELAY, TICKER_DELAY, MAX_DAYSOUT, MIN_DAYSOUT)
else:
    all_options = pd.read_csv('all_options_test.csv', index_col = 0)
    
#Calculate additional columns for all_options
if run_BuildOptionMetrics:
    all_options, clean_options = BuildOptionMetrics(all_options, FEE_SPREAD, HIDE_TICKERS, DATE_ONLY_FORMAT, DATE_TIME_FORMAT)
    write_file = 'all_options' + FILE_ID_TAG + '.csv'
    all_options.to_csv(write_file)
    print('Wrote all_options')
else:
    all_options = pd.read_csv('all_options_test.csv') # For testing only    


profiles = pd.read_csv('option_profiles.csv', index_col = 0)
float_columns = profiles.columns.tolist()
float_columns.remove('Active') # So, why have 'Active'? Maybe for later testing and run control?.
profiles[float_columns] = profiles[float_columns].astype(float).copy()

print('calling Bullets_STO')
Bullets_STO(clean_options, profiles)
print('Bullets_STO ran all the way.')


just read root_data
start Darqube access at:   2024-02-02 10:03:56.931006   Done in  1.2  minutes.
getting price for  AAL
getting price for  AAPL
getting price for  ABBV
getting price for  ABT
getting price for  ACLS
getting price for  ACRE
getting price for  AFL
getting price for  ADBE
getting price for  AFYA
getting price for  AIG
getting price for  AMD
getting price for  ALK
getting price for  AMGN
getting price for  AMP
getting price for  AMR
getting price for  AMT
getting price for  BNTX
getting price for  BRSP
getting price for  BSM
getting price for  CAG
getting price for  CAT
getting price for  CBRE
getting price for  CCI
getting price for  CEG
getting price for  CF
getting price for  CFR
getting price for  CHPT
getting price for  CL
getting price for  CLX
getting price for  CMA
getting price for  CMCSA
getting price for  CMG
getting price for  CPA
getting price for  AMZN
getting price for  AOS
getting price for  ARCC
getting price for  ARES
getting price for  ASC
getting price